# Common functions

* ```LoadAndParse<T>```: Loads a text file, line for line, removing empty lines, and passes it through the parse transformation. If no transformation is given, it will just return the raw line as a `String`. 
* ```LoadMap```: Loads a text file containing a rectangular array of integer values 0-9, and returns it as a two-dimensional integer array. No value testing is performed, so if the input file contains other chars than 0-9 unexpected results can be expected.
* ```VonNeumannNeighbourhood```: Returns an enumeration of all points in the 4-neighbourhood around the supplied point. If ```wrapAround``` is ```true``` then it is guaranteed to always return four ```Point```s, otherwise illegal values will be excluded when the ```location``` is along one of the edges of the map.
* ```MooreNeighbourhood```: Returns an enumeration of the 8 points directly or diagonally adjacent to the supplied location. Note: Points are _not_ supplied in a clock- or anticlockwise order. If the source location lies along the edges of the map, illegal positions will be excluded from the result set.
* ```Drawmap```: Debug method, nicely formats a 2d int array when every value is between 0 and 9. Also formats any other 2d int array but nowhere near as nicely.

# Common classes
* ```Point```: Simple record type of an X,Y int pair.

In [ ]:
public T[] LoadAndParse<T>(string path, Func<string, T> parse = null) {  
    parse = parse ?? (p => (T) Convert.ChangeType(p, typeof(T)));
    var rawInput = System.IO.File.ReadAllLines(path);
    return rawInput
    .Where(p=>!string.IsNullOrWhiteSpace(p))
    .Select(parse)
    .ToArray();
}

public int[,] LoadMap(string path) {
    var source = System.IO.File.ReadAllLines(path)
        .Where(p=>!String.IsNullOrWhiteSpace(p)).ToArray();
    var map = new int[source[0].Length, source.Length];

    for(int y = 0; y < source.Length; y++)
    for(int x = 0; x < source[0].Length; x++)
        map[x,y]=source[y][x]-'0';
    
    return map;
}

record Point(int X, int Y);

IEnumerable<Point> VonNeumannNeighbourhood(Point location, int maxX, int maxY, bool wrapAround = false) {
    if(location.X > 0) yield return location with { X = location.X - 1 };
    else if (wrapAround) yield return location with { X = maxX };
    if(location.Y > 0) yield return location with { Y = location.Y - 1 };
    else if (wrapAround) yield return location with { Y = maxY };
    if(location.X < maxX) yield return location with { X = location.X + 1 };
    else if (wrapAround) yield return location with { X = 0 };
    if(location.Y < maxY) yield return location with { Y = location.Y + 1 };
    else if (wrapAround) yield return location with { Y = 0 };
}

IEnumerable<Point> MooreNeighbourhood(Point location, int maxX, int maxY) {
    for (int x = location.X - 1; x <= location.X + 1; x++)
    for (int y = location.Y - 1; y <= location.Y + 1; y++){
        var probe = new Point(x,y);
        if(x >= 0 && y >= 0 && 
           x <= maxX && y <= maxY &&
           probe != location) 
            yield return probe;
    }
}

void DrawMap(int[,] map) {
    for(int y = 0; y < map.GetLength(1); y++) {
        string line = "";
        for(int x = 0; x < map.GetLength(0); x++) {
            line += map[x,y] + " ";
        }
        Console.WriteLine(line);
    }
}

# Day 1 - Sonar Sweep

Fairly straightforward implementation of moving sum, with the Part 1 puzzle being the special case of a moving average of 1 item.

Solution is strictly O(n)

The solution below leverages the fact that the window can be advanced with minimum calculations by subtracting the first item of the old window (which is about to exit the window) followed by adding the new element about to be added to the window. 

In this way, we can maintain a window of arbitrary size with just one addition and one subtraction pr. element.

In [ ]:
var input = LoadAndParse<int>(@"day1_input.txt");

Console.WriteLine($"Part 1: {CountIncrements(MovingSums(input, 1))}");
Console.WriteLine($"Part 2: {CountIncrements(MovingSums(input, 3))}");    

int CountIncrements(IEnumerable<int> source)
    => source.Zip(source.Skip(1)).Count(p=>p.Item1 < p.Item2);

IEnumerable<int> MovingSums(IEnumerable<int> source, int windowSize) {
    var initial = source.Take(windowSize).Sum();
    yield return initial;

    //Start a double iterator on the source collection, separated by windowSize elements
    foreach (var pair in source.Zip(source.Skip(windowSize))){

        //When advancing the window, remove the start element of the old window, and add the last element of the new window
        initial -= pair.Item1; 
        initial += pair.Item2;
        yield return initial;
    }
}

Part 1: 1564
Part 2: 1611


# Day 2: Dive!

In [ ]:
enum CommandDirection{ forward, down, up }
record Command (CommandDirection Direction, int Amount );
record Position(int Horizontal, int Depth);
record PositionWithAim(int Horizontal, int Depth, int Aim);

var input =  LoadAndParse<Command>(@"day2_input.txt", p=>{
    var particles = p.Split(" ");
    var direction = Enum.Parse<CommandDirection>(particles[0]);
    var amount = Int32.Parse(particles[1]);
    return new Command(direction, amount);
});

var part1position =new Position(0,0);

foreach(var command in input){
    part1position = command.Direction switch {
        CommandDirection.forward => part1position with { Horizontal = part1position.Horizontal + command.Amount },
        CommandDirection.down => part1position with { Depth = part1position.Depth + command.Amount },
        CommandDirection.up => part1position with { Depth = part1position.Depth - command.Amount },
        _=>part1position
    };
}

var part2position = new PositionWithAim(0,0,0);
foreach(var command in input){
    part2position = command.Direction switch {
        CommandDirection.forward => part2position with {
            Horizontal = part2position.Horizontal + command.Amount,
            Depth = part2position.Depth + part2position.Aim * command.Amount },
        CommandDirection.down => part2position with { Aim = part2position.Aim + command.Amount },
        CommandDirection.up => part2position with { Aim = part2position.Aim - command.Amount },
        _=>part2position
    };
}

Console.WriteLine($"Part 1: {part1position.Horizontal * part1position.Depth} ({part1position})");
Console.WriteLine($"Part 2: {part2position.Horizontal * part2position.Depth} ({part2position})");

Part 1: 1250395 (Position { Horizontal = 1909, Depth = 655 })
Part 2: 1451210346 (PositionWithAim { Horizontal = 1909, Depth = 760194, Aim = 655 })


# Day 3: Binary Diagnostic

In [ ]:
var input = LoadAndParse<string>(@"day3_input.txt");
var bitmask = (1<<input[0].Length)-1; 
int gamma = 0;

for(int i = 0; i < input[0].Length; i++){
    gamma <<= 1;
    if(input.Count(p=>p[i]=='1') > input.Length/2)
        gamma++;
}

var epsilon = ~gamma & bitmask;

Console.WriteLine($"Gamma: {gamma}");
Console.WriteLine($"Epsilon: {epsilon}");
Console.WriteLine($"Part 1: {gamma * epsilon}");

string Reduce(IEnumerable<string> source, char target)
{
    var limit = source.First().Length;
    for(int i = 0; i < limit; i++){            
        if(source.Count(p => p[i] == target) == source.Count(p => p[i] != target))
            source = source.Where(p=>p[i] == target).ToArray();
        else if(source.Count(p=>p[i]==target) > source.Count()/2)
            source = source.Where(p=>p[i] == '1').ToArray();
        else 
            source = source.Where(p=>p[i] == '0').ToArray();

        if(source.Count() == 1)
            return source.Single();
    }

    return source.Single();        
}

var O2 = Convert.ToInt32(Reduce(input, '1'), 2);
var CO2 = Convert.ToInt32(Reduce(input, '0'), 2);

Console.WriteLine($"O2: {O2}");
Console.WriteLine($"CO2: {CO2}");
Console.WriteLine($"Part 2: {O2 * CO2}");

Gamma: 2566
Epsilon: 1529
Part 1: 3923414
O2: 2919
CO2: 2005
Part 2: 5852595


# Day 4: Giant Squid

In [ ]:
class BingoBoard{
    int[,] Numbers = new int[5,5];
    bool[,] Marks = new bool[5,5];
    public static BingoBoard Parse(string[] source) {
        BingoBoard res = new();
        for(int y = 0; y < source.Length; y++) {
            var items = source[y]
                .Split(" ", StringSplitOptions.RemoveEmptyEntries)
                .Select(Int32.Parse).ToArray();
            for(int x = 0; x < items.Length; x++)
                res.Numbers[x,y]=items[x];
        }    
        return res;
    }   
    public void Mark(int entry) {
        for(int x = 0; x < 5; x++)
        for(int y = 0; y < 5; y++)
            if(Numbers[x,y] == entry)
                Marks[x,y] = true;            
    }
    public bool Bingo()
        => Enumerable.Range(0, 5).Any(x=>Enumerable.Range(0, 5).All(y => Marks[x,y]))
        || Enumerable.Range(0,5).Any(y=>Enumerable.Range(0, 5).All(x => Marks[x,y]));
    public int Score()
        => Numbers.Cast<int>().Zip(Marks.Cast<bool>()).Sum(p=>p.Item2?0:p.Item1);
}

List<BingoBoard> Boards = new();

var input = System.IO.File.ReadAllLines(@"day4_input.txt");
var numbers = input[0].Split(",").Select(Int32.Parse).ToArray();
var boards = input.Skip(1).Chunk(6).Select(p=>BingoBoard.Parse(p.Skip(1).ToArray())).ToArray();

BingoBoard firstWinner = default;
BingoBoard lastWinner = default;

foreach(var number in numbers) {
    foreach(var board in boards) board.Mark(number);    

    if(lastWinner != default) {
        Console.WriteLine($"The board to avoid scored {lastWinner.Score()} with final drawing of {number} for a result of {lastWinner.Score() * number}");        
        return;
    }

    if(firstWinner == default && boards.Count(p=>p.Bingo()) == 1) {
        firstWinner = boards.Single(p=>p.Bingo());   
        Console.WriteLine($"Winner winner, chicken dinner - final board score {firstWinner.Score()} with drawing {number} for a result of {firstWinner.Score() * number}");        
    }

    if(boards.Count(p=>!p.Bingo())==1)
        lastWinner = boards.Single(p=>!p.Bingo());
}

Winner winner, chicken dinner - final board score 766 with drawing 95 for a result of 72770
The board to avoid scored 296 with final drawing of 47 for a result of 13912


<null>

# Day 5: Hydrothermal Venture

In [ ]:
record LineSegment(Point From, Point To) {
    public bool IsAxisAligned => From.X == To.X || From.Y == To.Y;
};

var input = LoadAndParse<LineSegment>(@"day5_input.txt", p => {
    var sections = p.Split(" -> ")
    .Select(p=>p.Split(",").Select(Int32.Parse).ToArray())
    .Select(p=>new Point(p[0], p[1]))
    .OrderBy(p=>p.X).ThenBy(p=>p.Y) // Guarantee natural ordering, so the only thing we need to consider when updating the map is whether a 45 degree segment slopes up or down.
    .ToArray();

    return new LineSegment(sections[0], sections[1]);
});

int[,] buildMap(IEnumerable<LineSegment> lines){
    var xMax = lines.Max(p=>p.To.X);
    var yMax = lines.Max(p=>p.To.Y);

    var map = new int[xMax+1,yMax+1];

    foreach(var line in lines){
        int targets = line.From.X == line.To.X 
            ? line.To.Y - line.From.Y
            : line.To.X - line.From.X; //Also covers 45 degree diagonals
        for(int target = 0; target <= targets; target++){
            if(line.IsAxisAligned && line.From.X == line.To.X) map[line.From.X, line.From.Y + target]++;
            else if(line.IsAxisAligned) map[line.From.X + target, line.From.Y]++;
            else if(line.From.Y < line.To.Y) map[line.From.X + target, line.From.Y + target]++;
            else map[line.From.X + target, line.From.Y - target]++;
        }
    }
    return map;
}

var alignedMap = buildMap(input.Where(p=>p.IsAxisAligned));

Console.WriteLine();
var genericMap = buildMap(input);
Console.WriteLine(alignedMap.Cast<int>().Count(p=>p>1));
Console.WriteLine(genericMap.Cast<int>().Count(p=>p>1));

//Just for fun, lets try rendering the map - Unfortunately, no easter-egg I can spot, just a bunch of lines.
var m = genericMap.Cast<int>().Max();
var b = new System.Drawing.Bitmap(genericMap.GetLength(0), genericMap.GetLength(1));
for(int x = 0; x < genericMap.GetLength(0); x++) 
for(int y = 0; y < genericMap.GetLength(1); y++) {
    var lumen = (byte)(250 * genericMap[x,y] / m); 
    b.SetPixel(x, y,  System.Drawing.Color.FromArgb(lumen, lumen, lumen));
}
b.Save("day5_Map.bmp");


5306
17787


![image](day5_Map.bmp)

# --- Day 6: Lanternfish ---
Hint: Noone cares where the fish are, they'll still find eachother when it's time ;)

In [ ]:
var initialFish = System.IO.File.ReadAllText(@"day6_input.txt")
.Split(",", StringSplitOptions.RemoveEmptyEntries)
.Select(Int32.Parse)
.ToArray();

var school = new Dictionary<int, long>{[0]=0, [1]=0, [2]=0, [3]=0, [4]=0, [5]=0, [6]=0, [7]=0, [8]=0};
foreach (var fish in initialFish) school[fish]++;

void IncrementGeneration(){
    var fry = school[0];
    for (int j = 0; j < 8; j++)
        school[j] = school[j+1];
    school[6] += fry;
    school[8] = fry;        
}

for(int i = 0; i < 80; i++)
    IncrementGeneration();
Console.WriteLine(school.Values.Sum());

for(int i = 0; i < 256-80; i++)
    IncrementGeneration();
Console.WriteLine(school.Values.Sum());


350149
1590327954513


# --- Day 7: The Treachery of Whales ---

In [ ]:
var input = System.IO.File.ReadAllText(@"day7_input.txt")
    .Split(",").Select(Int32.Parse).ToArray();
int FuelUse(int position) => input.Sum(p=>Math.Abs(p-position));
int FuelUseCompound(int position) => input.Sum(p=>(Math.Abs(p-position)+1)  * Math.Abs(p-position) / 2);

var median = input.OrderBy(p=>p).Skip(input.Length/2).First();
Console.WriteLine($"Moving to position {median} uses {FuelUse(median)} fuel.");

var guess = median;
var compoundUse = FuelUseCompound(guess);
var seekDirection = compoundUse > FuelUseCompound(guess-1) ? -1 : 1;

while(compoundUse > FuelUseCompound(guess + seekDirection)){
    guess += seekDirection;
    compoundUse = FuelUseCompound(guess);
}

Console.WriteLine($"Stabilized at {guess} with a total compound use of {compoundUse}");

Moving to position 314 uses 323647 fuel.
Stabilized at 446 with a total compound use of 87640209


# --- Day 8: Seven Segment Search ---

In [ ]:
var input = LoadAndParse<(string[] Signals, string[] Output)>(@"day8_input.txt", p => {
  var signals = p.Split('|')[0].Split(" ", StringSplitOptions.RemoveEmptyEntries).ToArray();
  var outputs = p.Split('|')[1].Split(" ", StringSplitOptions.RemoveEmptyEntries).ToArray();
  return (signals, outputs);
});

Console.WriteLine(input.SelectMany(p=>p.Output).Count(p=>new[]{2, 3, 4, 7}.Contains(p.Length)));

Console.WriteLine(input.Sum(ResolveDisplayValue));

int ResolveDisplayValue((string[] Signals, string[] Output) state){
    var map = new HashSet<char>[10];
    map[1] = state.Signals.Single(p=>p.Length == 2).ToHashSet(); 
    map[7] = state.Signals.Single(p=>p.Length == 3).ToHashSet();    
    map[4] = state.Signals.Single(p=>p.Length == 4).ToHashSet();
    map[8] = state.Signals.Single(p=>p.Length == 7).ToHashSet();
    map[9] = state.Signals.Single(p=>p.Length == 6 && p.Intersect(map[1]).Count() == 2 && p.Intersect(map[4]).Count() == 4).ToHashSet();
    map[0] = state.Signals.Single(p=>p.Length == 6 && p.Intersect(map[1]).Count() == 2 && p.Intersect(map[9]).Count() != 6).ToHashSet();    
    map[6] = state.Signals.Single(p=>p.Length == 6 && p.Intersect(map[0]).Count() != 6 && p.Intersect(map[9]).Count() != 6).ToHashSet();
    map[2] = state.Signals.Single(p=>p.Length == 5 && p.Intersect(map[4]).Count() == 2).ToHashSet();
    map[3] = state.Signals.Single(p=>p.Length == 5 && p.Intersect(map[1]).Count() == 2 && p.Intersect(map[2]).Count() != 5).ToHashSet();
    map[5] = state.Signals.Single(p=>p.Length == 5 && p.Intersect(map[2]).Count() != 5 && p.Intersect(map[3]).Count() != 5).ToHashSet();

    int sum = 0;
    foreach (var digit in state.Output){
        sum *= 10;
        for (int i = 0; i < 10; i++)
            if(map[i].SetEquals(digit)) { sum+=i; break; }
    }
    return sum;
}

264
1063760


# --- Day 9: Smoke Basin ---

In [ ]:
var map = LoadMap(@"day9_input.txt");
var points = 
    from x in Enumerable.Range(0, map.GetLength(0))
    from y in Enumerable.Range(0, map.GetLength(1))
        select (new Point(x,y));

var xMax = map.GetLength(0) - 1;
var yMax = map.GetLength(1) - 1;
var lowPoints = points.Where(p=>VonNeumannNeighbourhood(p, xMax,yMax).All(q=>map[q.X, q.Y] > map[p.X, p.Y]));
Console.WriteLine(lowPoints.Sum(p=>map[p.X, p.Y]+1));

IEnumerable<Point> Basin(Point p){
    HashSet<Point> basin = new();
    Stack<Point> fringe = new();
    fringe.Push(p);
    while(fringe.Any()){
        var probe = fringe.Pop();
        if(map[probe.X, probe.Y] == 9) continue;
        if(basin.Contains(probe)) continue;
        basin.Add(probe);
        foreach(var neighbour in VonNeumannNeighbourhood(probe, xMax, yMax))
            fringe.Push(neighbour);        
    }
    return basin;
}

Console.WriteLine(lowPoints.Select(Basin).Select(p=>p.Count()).OrderByDescending(p=>p).Take(3).Aggregate(1, (p, q)=>p*q));

545
950600


# --- Day 10: Syntax Scoring ---

In [ ]:
var input = LoadAndParse<string>(@"day10_input.txt");
record ParserState(bool SyntaxError, char? FirstError, char[] Missing);
ParserState Parse(string input){
    Stack<char> state = new ();
    foreach(var next in input){
        if(new[]{'{', '[','<', '('}.Contains(next))
            state.Push(next);
        else {
            if(Math.Abs(state.Peek()-next)<3) //A bit of a cheat - opening and closing brackets of all four kinds are very close in the ascii table.
                state.Pop();
            else
                return new ParserState(true, next, Array.Empty<char>());        
        }
    }
    return new ParserState(false, null, state.ToArray());
}
//We can leverage that the state in the Parse method actually contains the opening brackets, not the missing closing.
//If we don't spend time reversing the brackets for part 2, everything fits in one switch.
int Cost(char c) => c switch { ')' => 3, ']' => 57, '}' => 1197, '>' => 25137, '(' => 1, '[' => 2, '{' => 3, '<' => 4, _ => throw new InvalidOperationException("What even is that?") };

var parserResults = input.Select(Parse).ToArray();
Console.WriteLine(parserResults.Where(p=>p.SyntaxError).Sum(p=>Cost(p.FirstError.Value)));
var scores = parserResults.Where(p=>!p.SyntaxError).Select(p=>p.Missing.Aggregate(0L, (p, q) => p * 5 + Cost(q))).OrderBy(p=>p).ToArray();
Console.WriteLine(scores[scores.Length/2]);

321237
2360030859


# --- Day 11: Dumbo Octopus ---

In [ ]:
var map = LoadMap(@"day11_input.txt");

int StepAndCountFlashes(int[,] map) {
    int flashCount = 0;
    for(int x = 0; x < map.GetLength(0); x++)
    for(int y = 0; y < map.GetLength(1); y++){
        map[x,y]++;
    }
    bool iterate = false;
    do {
        iterate = false;
        for(int x = 0; x < map.GetLength(0); x++)
        for(int y = 0; y < map.GetLength(1); y++){
            if(map[x,y] > 9){
                foreach (var neighbour in MooreNeighbourhood(new Point(x,y), map.GetLength(0) - 1, map.GetLength(1) - 1)){
                    if(map[neighbour.X, neighbour.Y] != 0) { //Don't increment 0s, they have already flashed.
                        map[neighbour.X, neighbour.Y]++;
                        iterate=true;
                    }
                }
                map[x,y] = 0;
                flashCount ++;
            }
        }
    } while (iterate);
    return flashCount;
}

int flashCount = 0;
for(int i = 0; i < 100; i++)
    flashCount += StepAndCountFlashes(map);   
Console.WriteLine(flashCount);

int steps = 101;
while(StepAndCountFlashes(map) != map.Length) steps++;
Console.WriteLine(steps);

1681
276


# --- Day 12: Passage Pathing ---

In [ ]:
class Graph {
    public record Node(string Name, List<Node> Neighbours);
    public Dictionary<string, Node> Nodes = new();
    
    public void Add(string from, string to) {
        if(!Nodes.ContainsKey(from)) Nodes[from] = new(from, new());
        if(!Nodes.ContainsKey(to)) Nodes[to] = new(to, new());
        Nodes[from].Neighbours.Add(Nodes[to]);
        Nodes[to].Neighbours.Add(Nodes[from]);
    }
    
    public IEnumerable<IEnumerable<string>> GetPaths(string from, string to, bool allowDuplicateVisit) {
        var visits = Nodes.Keys.ToDictionary(p => p, p => 0);
        if(allowDuplicateVisit) visits[from]++;

        return GetPathsInternal(from, to, allowDuplicateVisit, visits);       
    }

    protected IEnumerable<IEnumerable<string>> GetPathsInternal(string from, string to, bool allowDuplicateVisit, Dictionary<string, int> visits) {
        Dictionary<string, int> newVisits = new(visits);

        if(from == to) {
            yield return new[] { to };
            yield break;
        }

        var node = Nodes[from];
        if(!node.Name.All(Char.IsUpper))
            newVisits[from]++;
        
        if(newVisits.Count(p=>p.Value > 1) > (allowDuplicateVisit ? 2 : 0))
            yield break;
        
        foreach(var neighbour in node.Neighbours.Select(p=>p.Name)) {
            if(newVisits[neighbour] == 0 || allowDuplicateVisit && newVisits[neighbour] == 1) {
                foreach(var subPath in GetPathsInternal(neighbour, to, allowDuplicateVisit, newVisits))
                    yield return new[]{from}.Concat(subPath);
            }
        }
    }
}

Graph g = new();
foreach (var line in LoadAndParse<string[]>(@"day12_input.txt", p=>p.Split("-")))
    g.Add(line[0], line[1]);

Console.WriteLine(g.GetPaths("start", "end", false).Count());
Console.WriteLine(g.GetPaths("start", "end", true).Count());

3679
107395


# --- Day 13: Transparent Origami ---

This one really should be written in F# - it is pretty much all down to lambda calculus and collection transformation.

In [ ]:
record Fold(char Axis, int Offset){
    public Point Transform(Point p) => Axis switch {
        'x' => p.X > Offset ? p with { X = Offset - (p.X - Offset) } : p,
        'y' => p.Y > Offset ? p with { Y = Offset - (p.Y - Offset) } : p,
        _ => throw new ArgumentException("Cannot comprehend axis " + Axis)
    };
};
List<Point> dots = new();
List<Fold> folds = new();

foreach(var line in LoadAndParse<string>(@"day13_input.txt")){
    if(line.Contains(",")){
        var coord = line.Split(",", StringSplitOptions.RemoveEmptyEntries).Select(Int32.Parse).ToArray();
        dots.Add(new(coord[0],coord[1]));
    }
    else if (line.Contains("fold along")){
        folds.Add(new(line[11], Int32.Parse(line.Substring(13))));
    }    
}

Console.WriteLine($"After the first fold, there are {dots.Select(folds[0].Transform).Distinct().Count()} distinct dots (out of {dots.Count()} starting dots) left.");
var fullTransform = folds.Select(p=>p.Transform).Aggregate((a, b) => p => (b(a(p))));
var message = dots.Select(fullTransform).ToHashSet();
Point bounds = new(message.Max(p=>p.X), message.Max(p=>p.Y));

Console.WriteLine("The message decodes as: ");
for(var y = 0; y <= bounds.Y; y++) {
    string line = "";
    for(int x = 0; x <= bounds.X; x++) {
        if(message.Contains(new(x,y))) line += '*';
        else line += ' ';
    }
    Console.WriteLine(line);
}

After the first fold, there are 684 distinct dots (out of 846 starting dots) left.
The message decodes as: 
  ** ***  **** ***  *     **  *  * *  *
   * *  *    * *  * *    *  * * *  *  *
   * *  *   *  ***  *    *    **   ****
   * ***   *   *  * *    * ** * *  *  *
*  * * *  *    *  * *    *  * * *  *  *
 **  *  * **** ***  ****  *** *  * *  *


# --- Day 14: Extended Polymerization ---

In [ ]:
var input = LoadAndParse<string>(@"day14_input.txt");
var template = input.First();
var rules = input.Skip(1).ToDictionary(p=>p.Substring(0, 2), p=>p[6]);

LinkedList<char> polymer = new(template);

void Step() {
    var iterator = polymer.First.Next;
    while(iterator != null) {
        if(rules.TryGetValue($"{iterator.Previous.Value}{iterator.Value}", out var insertion))
            polymer.AddAfter(iterator.Previous, insertion);
        iterator = iterator.Next;
    }
}

//naïve implementation for part 1 - just scrapes by but real short and neat.
for(int step = 0; step < 10; step++) Step();
var elements = polymer.GroupBy(p=>p).ToDictionary(p=>p.Key, p=>p.LongCount());
Console.WriteLine(elements.Max(p=>p.Value) - elements.Min(p=>p.Value));

//for part 2 we need to be a bit more clever - could most likely be a lot more short and neat still..
var digrams = Enumerable.Range(0, template.Length-1)
    .Select(p=>template.Substring(p, 2))
    .GroupBy(p=>p)
    .ToDictionary(p=>p.Key, p=>p.LongCount());
foreach(var rule in rules){
    if(!digrams.ContainsKey(rule.Key))
        digrams[rule.Key] = 0;
}

for(int i = 0; i < 40; i++){
    var temp = new Dictionary<string, long>(digrams);
    foreach(var rule in rules){
        if(digrams.ContainsKey(rule.Key)){
            temp[rule.Key] -= digrams[rule.Key];
            temp[$"{rule.Key[0]}{rule.Value}"] += digrams[rule.Key];
            temp[$"{rule.Value}{rule.Key[1]}"] += digrams[rule.Key];
        }
    }
    digrams = temp;
}

elements = new Dictionary<char, long>();
foreach (var digram in digrams){
    foreach (var element in digram.Key){
        if(!elements.ContainsKey(element)) elements[element] = 0;
        elements[element]+= digram.Value;
    }
}

//Every other element is double counted except the first and last since they don't have a left or right neighbour.
elements[template[0]]++;
elements[template.Last()]++;

Console.WriteLine((elements.Max(p=>p.Value) - elements.Min(p=>p.Value))/2);


3284
4302675529689
